# Load Model for Predictions & Evaluate Predictions

In [1]:
import sys
import pandas as pd
import numpy as np

In [2]:
sys.path.append("../../../python-package")
import lightgbm as lgb

## I. Read Model and Data

In [3]:
model = lgb.Booster(model_file='LightGBM_model.txt')

In [77]:
feature_names = model.feature_name()
feature_names[:5]

['Column_0', 'Column_1', 'Column_2', 'Column_3', 'Column_4']

In [78]:
len(feature_names)

301

In [6]:
def parse_line(line):
    
    linesp = line.split()
    score = linesp[0]
    feats = linesp[1:]
    feats = {el.split(":")[0]:float(el.split(":")[1]) for el in feats}
    feats['score'] = float(score)
    
    return feats
    
def parse_test_file(filename, query_filename=None):
    
    with open(filename) as f:
        lines = f.readlines()
        
    test = pd.DataFrame.from_dict([parse_line(line) for line in lines])
    cols = test.columns.to_list()
    cols.remove("score")
    cols_sorted = sorted(cols, key=lambda txt: int(txt))
    
    test = test[['score'] + cols_sorted]
    
    if not query_filename: return test
    
    queries = pd.read_csv(query_filename, header=None).astype(int)
    #queries = queries.rename(columns={0:"qid"})
    test_q = test.copy()

    query_col = []
    # set arbitrary but unique query id
    for i, q in enumerate(queries[0]):
        query_col.extend([i]*q)
    len(query_col)

    test_q.insert(0, 'qid', query_col)
    
    return test_q
    
    
    

In [118]:
test = parse_test_file("rank.test", "../rank.test.query")
test.head()

,qid,score,1,2,6,7,8,9,10,11,...,289,290,291,292,294,295,297,298,299,300
0,0,2.0,0.74,NaN,0.87,NaN,0.75,0.8,NaN,0.88,...,NaN,0.86,NaN,NaN,NaN,NaN,0.36,NaN,0.99,0.70
1,0,3.0,0.74,NaN,0.81,NaN,0.60,0.8,NaN,0.88,...,NaN,0.86,NaN,NaN,NaN,NaN,0.36,NaN,NaN,0.43
2,0,2.0,0.74,NaN,0.80,NaN,0.60,NaN,NaN,0.88,...,NaN,0.98,NaN,NaN,NaN,NaN,0.36,NaN,NaN,0.70
3,0,0.0,0.74,NaN,0.84,0.81,0.66,0.8,NaN,0.88,...,NaN,0.95,NaN,NaN,NaN,NaN,0.36,NaN,0.88,0.55
4,0,2.0,0.74,NaN,0.91,0.81,0.83,0.8,NaN,0.88,...,NaN,0.90,NaN,NaN,NaN,NaN,0.36,NaN,0.97,0.43


In [119]:
test.shape

(768, 219)

In [120]:
def adjust_features_for_model(model, df, extra_cols=['qid', 'score']):
    
    feat_cols = [col for col in df.columns if col not in extra_cols]
    
    all_feats = pd.DataFrame(columns=model.feature_name())

    for col in feat_cols:
        col_name = f"Column_{col}"
        #print(col_name)
        all_feats[col_name] = df[col]
    all_feats = all_feats.astype(float)
        
    for col in extra_cols:
        all_feats.insert(0, col, df[col])
        
    return all_feats

In [121]:
test = adjust_features_for_model(model, test)

/Users/christian/opt/anaconda3/envs/searchrec/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  exec(code_obj, self.user_global_ns, self.user_ns)


In [122]:
test.head()

,score,qid,Column_0,Column_1,Column_2,Column_3,Column_4,Column_5,Column_6,Column_7,...,Column_291,Column_292,Column_293,Column_294,Column_295,Column_296,Column_297,Column_298,Column_299,Column_300
0,2.0,0,NaN,0.74,NaN,NaN,NaN,NaN,0.87,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.36,NaN,0.99,0.70
1,3.0,0,NaN,0.74,NaN,NaN,NaN,NaN,0.81,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.36,NaN,NaN,0.43
2,2.0,0,NaN,0.74,NaN,NaN,NaN,NaN,0.80,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.36,NaN,NaN,0.70
3,0.0,0,NaN,0.74,NaN,NaN,NaN,NaN,0.84,0.81,...,NaN,NaN,NaN,NaN,NaN,NaN,0.36,NaN,0.88,0.55
4,2.0,0,NaN,0.74,NaN,NaN,NaN,NaN,0.91,0.81,...,NaN,NaN,NaN,NaN,NaN,NaN,0.36,NaN,0.97,0.43


## II. Predictions

In [123]:
preds = model.predict(test[test.columns[2:]])

In [124]:
preds[:10]

array([ 0.31450323, -0.26806001, -0.04428472, -0.01040121, -0.11274014,
       -0.20809923, -0.25287134,  0.26225523, -0.29988372, -1.10203005])

In [125]:
cli_preds = pd.read_csv("predict_result_test.txt", header=None)[0]

In [126]:
cli_preds[:10]

0    0.314503
1   -0.268060
2   -0.044285
3   -0.010401
4   -0.112740
5   -0.208099
6   -0.252871
7    0.262255
8   -0.299884
9   -1.102030
Name: 0, dtype: float64

## III. Evaluate Predictions

In [127]:
test.insert(2, 'score_pred', preds)

In [128]:
test.head()

,score,qid,score_pred,Column_0,Column_1,Column_2,Column_3,Column_4,Column_5,Column_6,...,Column_291,Column_292,Column_293,Column_294,Column_295,Column_296,Column_297,Column_298,Column_299,Column_300
0,2.0,0,0.314503,NaN,0.74,NaN,NaN,NaN,NaN,0.87,...,NaN,NaN,NaN,NaN,NaN,NaN,0.36,NaN,0.99,0.70
1,3.0,0,-0.268060,NaN,0.74,NaN,NaN,NaN,NaN,0.81,...,NaN,NaN,NaN,NaN,NaN,NaN,0.36,NaN,NaN,0.43
2,2.0,0,-0.044285,NaN,0.74,NaN,NaN,NaN,NaN,0.80,...,NaN,NaN,NaN,NaN,NaN,NaN,0.36,NaN,NaN,0.70
3,0.0,0,-0.010401,NaN,0.74,NaN,NaN,NaN,NaN,0.84,...,NaN,NaN,NaN,NaN,NaN,NaN,0.36,NaN,0.88,0.55
4,2.0,0,-0.112740,NaN,0.74,NaN,NaN,NaN,NaN,0.91,...,NaN,NaN,NaN,NaN,NaN,NaN,0.36,NaN,0.97,0.43


### Calculate NDCG@5 Score and confirm with Modeling Metrics

In [129]:
from sklearn.metrics import ndcg_score

In [137]:
def calc_ndcg_per_query(grp, k):
    
    if grp['score'].nunique()>1:
        return ndcg_score([grp['score'].values], [grp['score_pred'].values], k=k)
    else:
        # if query has only one doc
        return np.nan

In [131]:
ndcg_per_query = test.groupby("qid").apply(calc_ndcg_per_query, k=5)

In [132]:
ndcg_per_query.mean()

0.7015621882967505

> NDCG@5 is in agreement with best metric value in evaluation during training (Evaluation_ReadLogs) 

In [133]:
test['score_pred'] = cli_preds

In [134]:
test.groupby("qid").apply(calc_ndcg_per_query, k=5).mean()

0.7015621882967505

In [141]:
test['score'].unique()

array([2., 3., 0., 1., 4.])

> In agreement between cli prediction and model python sdk prediction

With random results:

In [138]:
np.random.seed(10)
test['score_pred'] = np.random.random(test.shape[0])

In [136]:
test.groupby("qid").apply(calc_ndcg_per_query, k=5).mean()

0.5635562126414108